# Python’s Asyncio

Concurrency is the ability of a program to manage multiple tasks at the same time, making efficient use of resources like the CPU. It doesn't mean tasks are happening simultaneously but rather that the program can switch between tasks quickly, so it seems like they are running together. This is crucial for operations that involve waiting, like reading files, fetching data from the internet, or interacting with a database. Without concurrency, a program would waste time sitting idle during these waiting periods.

We need concurrency to improve the performance and responsiveness of applications, especially those that handle multiple I/O-bound tasks. For example, a web server can handle requests from many users at once without making each user wait for others to finish. We use concurrency in scenarios where tasks can be paused and resumed, allowing the program to keep working on other tasks in the meantime. This approach is common in network services, real-time data processing, and user interface applications, where maintaining a smooth, responsive experience is key.

In [2]:
import nest_asyncio
import asyncio
import time
import aiohttp
import aiofiles

# Allow asyncio to run in Jupyter Notebooks
nest_asyncio.apply()

## Python Asyncio Concepts

### Event Loop

The event loop is the central hub of Python's `asyncio` framework, responsible for managing the execution of asynchronous tasks. It continually checks for tasks that are ready to run and dispatches them accordingly, ensuring that no task blocks the others unnecessarily. When a task is paused because it's waiting for something (like a network response or a timer), the event loop picks up another task that is ready to run, allowing the program to make progress even when some tasks are idle. Think of the event loop as a director in a theater, ensuring that each actor (task) knows when to step on stage and when to step back, allowing the show (your program) to flow smoothly without delays.

In practical terms, the event loop keeps track of all the asynchronous tasks, managing when they start, pause, and finish. It efficiently handles these tasks by making sure the CPU isn't wasted on tasks that are just waiting. This makes it an essential part of writing non-blocking, concurrent programs, where you can run many I/O-bound operations simultaneously without waiting for each to complete one by one.

### Coroutines

Coroutines are the building blocks of asynchronous programming in Python, defined using the `async def` syntax. Unlike regular functions, coroutines can pause their execution at certain points (using `await`) and yield control back to the event loop. This allows other tasks to run while the coroutine is waiting, making the overall execution more efficient. Coroutines are like actors in a play who pause their performance when they need to, allowing others to take the stage. Once the necessary condition is met (like the completion of an I/O operation), the coroutine resumes its execution from where it left off.

For example, consider a coroutine that needs to wait for a web page to load. Instead of blocking the entire program, it pauses and lets other tasks (like loading another web page) run. This makes coroutines ideal for tasks that involve waiting, as they keep the event loop active and other tasks moving forward, ensuring that your application remains responsive.

### Futures and Tasks

A Future in `asyncio` represents a result that hasn't been computed yet but will be available at some point in the future. It's essentially a placeholder that allows the program to continue running other tasks while waiting for a particular result. Futures are low-level objects that are usually handled by the event loop, and they're not often created directly by developers when writing high-level asynchronous code. However, understanding them is crucial when dealing with more complex async operations, as they provide a mechanism to manage results that aren’t immediately available.

Tasks, on the other hand, are a higher-level abstraction built on top of Futures. When you schedule a coroutine to run with `asyncio.create_task()`, it is wrapped in a Task, which is then managed by the event loop. Tasks are what you usually work with in `asyncio` to run coroutines concurrently. They ensure that the coroutine's execution is tracked and that its result (or exception) is captured and handled once it's done. Tasks are the practical way to handle multiple operations that can run independently, making them an essential tool for building responsive and efficient applications.

## Getting Started with Asyncio

### Basic Asyncio Example: Printing a Message

In [3]:
async def say_hello_async():
    await asyncio.sleep(2)
    print("Hello, Async World!")

await say_hello_async()

Hello, Async World!


### Running Multiple Tasks Concurrently

Asyncio shines when doing multiple tasks concurrently.

In [4]:
async def say_hello_async():
    await asyncio.sleep(2)
    print("Hello, Async World!")

async def do_something_else():
    print("Doing something else...")
    await asyncio.sleep(1)
    print("Finished something else.")

await asyncio.gather(say_hello_async(), do_something_else())

Doing something else...
Finished something else.
Hello, Async World!


[None, None]

### Synchronous vs. Asynchronous HTTP Requests

#### Synchronous Example:

In [5]:
import requests

def fetch_sync(url):
    return requests.get(url).text

start_time = time.time()
page1 = fetch_sync('http://example.com')
page2 = fetch_sync('http://example.org')
print(f"Done in {time.time() - start_time} seconds")

Done in 0.45296788215637207 seconds


#### Asynchronous Example:

In [6]:
async def fetch_async(url, session):
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        page1 = asyncio.create_task(fetch_async('http://example.com', session))
        page2 = asyncio.create_task(fetch_async('http://example.org', session))
        await asyncio.gather(page1, page2)

start_time = time.time()
await main()
print(f"Done in {time.time() - start_time} seconds")

Done in 0.22742891311645508 seconds


### Asynchronous File Reading with aiofiles

Use `aiofiles` to read files without blocking the event loop.

In [7]:
async def read_file_async(filepath):
    async with aiofiles.open(filepath, 'r') as file:
        return await file.read()

async def read_all_async(filepaths):
    tasks = [read_file_async(filepath) for filepath in filepaths]
    return await asyncio.gather(*tasks)

filepaths = ['data/large_log_file.txt', 'data/processed_errors.txt']
data = await read_all_async(filepaths)
print(data)

['2024-05-21 12:00:00 - INFO - This is an informational message.\n2024-05-21 12:01:00 - ERROR - This is an error message.\n2024-05-21 12:02:00 - WARNING - This is a warning message.\n2024-05-21 12:03:00 - ERROR - Another error occurred.\n2024-05-21 12:04:00 - INFO - Another informational message.\n2024-05-21 12:05:00 - ERROR - Yet another error message.\n2024-05-21 12:06:00 - INFO - More info.\n2024-05-21 12:07:00 - ERROR - Error with additional details.\n2024-05-21 12:08:00 - WARNING - Warning with additional details.\n2024-05-21 12:09:00 - ERROR - Final error message.', '21-05-2024 12:01:00 - ERROR - This is an error message.\n21-05-2024 12:03:00 - ERROR - Another error occurred.\n21-05-2024 12:05:00 - ERROR - Yet another error message.\n21-05-2024 12:07:00 - ERROR - Error with additional details.\n21-05-2024 12:09:00 - ERROR - Final error message.\n']


### Running Synchronous Functions Asynchronously

Sometimes, you have to run blocking code (like old libraries) in an async environment.

In [8]:
def sync_task():
    print("Starting a long sync task...")
    time.sleep(5)
    print("Finished the sync task.")

async def async_wrapper():
    loop = asyncio.get_running_loop()
    await loop.run_in_executor(None, sync_task)

await async_wrapper()

Starting a long sync task...
Finished the sync task.


### Managing Futures in Asyncio

A Future represents a result that isn’t available yet.

In [9]:
async def async_operation(future, data):
    await asyncio.sleep(1)
    if data == "success":
        future.set_result("Operation succeeded")
    else:
        future.set_exception(RuntimeError("Operation failed"))

future = asyncio.Future()
await async_operation(future, "success")
if future.done():
    print(future.result())

Operation succeeded


## References

1. [Python’s AsyncIO: An Introduction](https://realpython.com/async-io-python/)

2. [Mastering Pythons Asyncio: A Practical Guide.](https://medium.com/@moraneus/mastering-pythons-asyncio-a-practical-guide-0a673265cf04)